In [14]:
import pandas as pd
from ast import literal_eval
import seaborn as sns
import numpy as np


movies_metadata = pd.read_csv("data/movies_metadata.csv")
links_small  = pd.read_csv("data/links_small.csv")
movies_keywords  = pd.read_csv("data/keywords.csv")

links_small = links_small[links_small['tmdbId'].notnull()]["tmdbId"].astype('int')
movies_metadata_small = movies_metadata[movies_metadata['id'].isin(links_small.astype('str'))]

movies_metadata_small.shape




C:\Users\Admin\AppData\Local\Temp\ipykernel_2592\2495644953.py:7: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd.read_csv("data/movies_metadata.csv")


(9099, 24)

In [15]:
movies_metadata_small['id'] # object
movies_keywords['id'] = movies_keywords['id'].astype('str') # int -> object


In [16]:
movies_metadata_small = movies_metadata_small.merge(movies_keywords, on='id')

In [ ]:
movies_metadata_small.head() # keywords

In [18]:
movies = movies_metadata_small[['title', 'genres','keywords','overview','popularity', 'release_date']]

In [ ]:
movies.head()

In [ ]:
movies['str_genres'] = movies['genres'].fillna('[]') \
                    .apply(literal_eval) \
                    .apply(lambda x: sorted(i['name'] for i in x ) if isinstance(x,list) else []) \
                    .apply(lambda x: ", ".join(x) if len(x) > 0 else None)



In [22]:
movies.head()

,title,genres,keywords,overview,popularity,release_date,str_genres
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,"Animation, Comedy, Family"
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 10090, 'name': 'board game'}, {'id': 1...",When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,"Adventure, Family, Fantasy"
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...",A family wedding reignites the ancient feud be...,11.7129,1995-12-22,"Comedy, Romance"
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'id': 818, 'name': 'based on novel'}, {'id':...","Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,"Comedy, Drama, Romance"
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...",Just when George Banks has recovered from his ...,8.387519,1995-02-10,Comedy


In [23]:
movies['str_keywords'] = movies['keywords'].fillna('[]') \
                    .apply(literal_eval) \
                    .apply(lambda x: sorted(i['name'] for i in x ) if isinstance(x,list) else []) \
                    .apply(lambda x: ", ".join(x) if len(x) > 0 else None)

C:\Users\Admin\AppData\Local\Temp\ipykernel_2592\4060563036.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['str_keywords'] = movies['keywords'].fillna('[]') \


In [24]:
movies['str_keywords'].head()

0    boy, boy next door, friends, friendship, jealo...
1    based on children's book, board game, disappea...
2    best friend, duringcreditsstinger, fishing, ol...
3    based on novel, chick flick, divorce, interrac...
4    aging, baby, confidence, contraception, daught...
Name: str_keywords, dtype: object

In [26]:
def get_genres_keywords_overview(row):
    return f"""
    genres:
    {row['str_genres']}

    
    keywords:
    {row['str_keywords']}

    overview:
    {row['overview']}

    """


In [27]:
movies['genres_keywords_overview'] = movies.apply(get_genres_keywords_overview,axis=1)

C:\Users\Admin\AppData\Local\Temp\ipykernel_2592\2575982863.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['genres_keywords_overview'] = movies.apply(get_genres_keywords_overview,axis=1)


In [30]:
movies['genres_keywords_overview'].iloc[0]

"\n    genres:\n    Animation, Comedy, Family\n\n\n    keywords:\n    boy, boy next door, friends, friendship, jealousy, new toy, rivalry, toy, toy comes to life\n\n    overview:\n    Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.\n\n    "

In [31]:
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')
movies['year'] = movies['release_date'].dt.year

movies['popularity'] = movies['popularity'].astype(float)
movies['popularity_log'] = np.log(movies['popularity'])

C:\Users\Admin\AppData\Local\Temp\ipykernel_2592\2474586242.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')
C:\Users\Admin\AppData\Local\Temp\ipykernel_2592\2474586242.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['year'] = movies['release_date'].dt.year
C:\Users\Admin\AppData\Local\Temp\ipykernel_2592\2474586242.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

In [32]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9151 entries, 0 to 9150
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   title                     9151 non-null   object        
 1   genres                    9151 non-null   object        
 2   keywords                  9151 non-null   object        
 3   overview                  9139 non-null   object        
 4   popularity                9151 non-null   float64       
 5   release_date              9151 non-null   datetime64[ns]
 6   str_genres                9116 non-null   object        
 7   str_keywords              8380 non-null   object        
 8   genres_keywords_overview  9151 non-null   object        
 9   year                      9151 non-null   int32         
 10  popularity_log            9151 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int32(1), object(7)
memory usage: 750.8+ KB


In [33]:
movies = movies.dropna().reset_index(drop=True)

In [34]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8368 entries, 0 to 8367
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   title                     8368 non-null   object        
 1   genres                    8368 non-null   object        
 2   keywords                  8368 non-null   object        
 3   overview                  8368 non-null   object        
 4   popularity                8368 non-null   float64       
 5   release_date              8368 non-null   datetime64[ns]
 6   str_genres                8368 non-null   object        
 7   str_keywords              8368 non-null   object        
 8   genres_keywords_overview  8368 non-null   object        
 9   year                      8368 non-null   int32         
 10  popularity_log            8368 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int32(1), object(7)
memory usage: 686.6+ KB


## LLM 기반 Contents Based Filtering

In [ ]:
from transformers import BertTokenizer,BertModel
import torch




In [40]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [41]:
sentence = "Transformers are amazing!"

tokens = tokenizer.tokenize(sentence)
tokens

['transformers', 'are', 'amazing', '!']